In [ ]:
using BenchmarkTools
using Revise

In [ ]:
using CSV
using DataFrames
using JSON

using DataIO
using GCTGMT
using Normalization
using Support

using FeatureSetEnrichment

In [ ]:
#
gene_set_directory_path = "/Users/kwat/Downloads/gene_sets/"

#
name_to_set_to_element_ = Dict()

for name in readdir(gene_set_directory_path)
    
    #
    if name[1] == '.'

        continue

    end

    #
    name_to_set_to_element_[name] = read_gmt(joinpath(gene_set_directory_path, name))
    
end

In [ ]:
#
benchmarks_directory_path = "/Users/kwat/Downloads/score_and_set/"

#
method_ = Vector{String}()

for m in keys(score_set_new(["a", "b"], [-1.0, 1.0], ["a"]; plot = false))

    push!(method_, string(m, " extreme"), string(m, " area"))

end

n_method = length(method_)

In [ ]:
#
benchmark_ = Vector{String}()

benchmark_x_method_rows = Vector{Vector{Float64}}()

;

In [ ]:
for name in readdir(benchmarks_directory_path)
    
    #
    if name[1] == '.'

        continue

    end

    println(name)
    
    #
    benchmark_directory_path = joinpath(benchmarks_directory_path, name)

    element_, score_ =
        eachcol(read_data(joinpath(benchmark_directory_path, "gene_x_score.tsv")))

    json_dict = JSON.parse(open(joinpath(benchmark_directory_path, "gene_set.json")))

    #
    set_to_element_ = Dict{String, Vector{String}}()
    
    for gmt_path in json_dict["gene_sets_tested"]
        
        merge!(set_to_element_, name_to_set_to_element_[replace(gmt_path, "../gene_sets/" => "")])
        
    end

    #
    set_ = sort(collect(keys(set_to_element_)))

    n_set = length(set_)

    println(n_set, " set.")

    #
    set_to_method_to_result = score_set_new(element_, score_, set_to_element_)

    #
    set_x_method = Matrix{Float64}(undef, n_set, n_method)

    for (set_i, (set, method_to_result)) in enumerate(set_to_method_to_result)

        set_x_method_row = []

        for result in values(method_to_result)

            append!(set_x_method_row, result[2:3])

        end

        set_x_method[set_i, :] = set_x_method_row

    end

    #
    for set in json_dict["gene_sets_positive"]

        #
        print("    ")

        if in(set, set_)

            println(set)

        else

            println(set, " (missing)")

            continue

        end

        #
        push!(benchmark_, string(name, '.', set))

        #
        benchmark_x_method_row = Vector{Float64}()

        for (method_i, set_score_) in enumerate(eachcol(set_x_method))
            
            #
            is_negative_ = set_score_ .< 0

            is_positive_ = 0 .<= set_score_

            #
            set_negative_ = set_[is_negative_]

            set_positive_ = set_[is_positive_]

            #
            negative_ = .-normalize(.-set_score_[is_negative_], "1224") / sum(is_negative_)

            positive_ = normalize(set_score_[is_positive_], "1224") / sum(is_positive_)

            #
            i_negative = findfirst(set_negative_ .== set)

            i_positive = findfirst(set_positive_ .== set)

            @assert (i_negative !== nothing || i_positive !== nothing) && !(i_negative !== nothing && i_positive !== nothing)

            if i_negative !== nothing

                benchmark_score = negative_[i_negative]

            elseif i_positive !== nothing

                benchmark_score = positive_[i_positive]

            end

            push!(benchmark_x_method_row, benchmark_score)

        end

        push!(benchmark_x_method_rows, benchmark_x_method_row)

    end

end

In [ ]:
benchmark_x_method = Matrix{Float64}(undef, length(benchmark_x_method_rows), n_method)

for (i, v) in enumerate(benchmark_x_method_rows)

    benchmark_x_method[i, :] = v

end

benchmark_x_method = DataFrame(benchmark_x_method, method_)

insertcols!(benchmark_x_method, 1, Symbol("Benchmark") => benchmark_)

In [ ]:
benchmark_x_method_path = "benchmark_x_method.tsv"

In [ ]:
CSV.write(benchmark_x_method_path, benchmark_x_method)

In [ ]:
benchmark_x_method = read_data(benchmark_x_method_path)

In [ ]:
sort_like((median.(eachcol(benchmark_x_method[:, 2:end])), names(benchmark_x_method)))[2]